# 先物データの可視化分析

LME、SHFE、CMXの銅先物データをモダンでおしゃれなビジュアルで分析します。

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings

# プロジェクトのルートディレクトリをPythonパスに追加
project_root = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.insert(0, project_root)

from config.database_config import get_connection_string

warnings.filterwarnings('ignore')

# カスタムカラーパレット
COLORS = {
    'primary': '#2E86AB',      # Deep Blue
    'secondary': '#A23B72',    # Purple Pink
    'accent': '#F18F01',       # Orange
    'success': '#C73E1D',      # Red Orange
    'dark': '#2D3436',         # Dark Gray
    'light': '#F7F7F7',        # Light Gray
    'gradient': ['#667eea', '#764ba2', '#f093fb', '#f5576c', '#4facfe', '#00f2fe']
}

# Plotlyテーマ設定
plotly_template = dict(
    layout=go.Layout(
        font=dict(family="Arial, sans-serif", size=12, color=COLORS['dark']),
        plot_bgcolor=COLORS['light'],
        paper_bgcolor='white',
        margin=dict(l=60, r=30, t=60, b=60),
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial")
    )
)

print("📊 ライブラリのインポートが完了しました")

## データベース接続とデータ取得

In [ ]:
def get_futures_data(conn, days=90):
    """先物データを取得"""
    query = f"""
    SELECT 
        p.TradeDate,
        m.MetalCode,
        m.ExchangeCode,
        t.TenorTypeName,
        p.SettlementPrice,
        p.Volume,
        p.OpenInterest,
        CASE 
            WHEN t.TenorTypeName LIKE 'Generic 1%' THEN 1
            WHEN t.TenorTypeName LIKE 'Generic 2%' THEN 2
            WHEN t.TenorTypeName LIKE 'Generic 3%' THEN 3
            WHEN t.TenorTypeName LIKE 'Generic 4%' THEN 4
            WHEN t.TenorTypeName LIKE 'Generic 5%' THEN 5
            WHEN t.TenorTypeName LIKE 'Generic 6%' THEN 6
            WHEN t.TenorTypeName LIKE 'Generic 7%' THEN 7
            WHEN t.TenorTypeName LIKE 'Generic 8%' THEN 8
            WHEN t.TenorTypeName LIKE 'Generic 9%' THEN 9
            WHEN t.TenorTypeName LIKE 'Generic 10%' THEN 10
            WHEN t.TenorTypeName LIKE 'Generic 11%' THEN 11
            WHEN t.TenorTypeName LIKE 'Generic 12%' THEN 12
            ELSE 0
        END as TenorNumber
    FROM T_CommodityPrice p
    INNER JOIN M_Metal m ON p.MetalID = m.MetalID
    INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
    WHERE 
        t.TenorTypeName LIKE 'Generic%Future%'
        AND p.TradeDate >= DATEADD(day, -{days}, GETDATE())
        AND p.SettlementPrice IS NOT NULL
    ORDER BY p.TradeDate DESC, m.ExchangeCode, t.TenorTypeID
    """
    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy")
        df = pd.read_sql(query, conn)
    
    df['TradeDate'] = pd.to_datetime(df['TradeDate'])
    return df

# データベース接続
conn = pyodbc.connect(get_connection_string())
print("✅ データベースに接続しました")

# データ取得
futures_df = get_futures_data(conn, days=90)
print(f"📈 {len(futures_df):,}件のデータを取得しました")
print(f"📅 期間: {futures_df['TradeDate'].min().strftime('%Y-%m-%d')} ～ {futures_df['TradeDate'].max().strftime('%Y-%m-%d')}")

# ExchangeCodeがNoneでないものだけを抽出
exchanges = [str(x) for x in futures_df['ExchangeCode'].unique() if x is not None]
if exchanges:
    print(f"🏢 取引所: {', '.join(exchanges)}")
else:
    print("🏢 取引所: データなし")

## 1. 先物カーブの3D可視化

In [ ]:
# LMEデータのみ抽出
lme_data = futures_df[futures_df['ExchangeCode'] == 'LME'].copy()

# 最新20日分のデータでピボット
recent_dates = lme_data['TradeDate'].unique()[:20]
lme_recent = lme_data[lme_data['TradeDate'].isin(recent_dates)]

# ピボットテーブル作成
pivot_data = lme_recent.pivot_table(
    values='SettlementPrice',
    index='TradeDate',
    columns='TenorNumber',
    aggfunc='mean'
).sort_index(ascending=False)

# 3Dサーフェスプロット
fig = go.Figure(data=[go.Surface(
    z=pivot_data.values,
    x=pivot_data.columns,
    y=pivot_data.index,
    colorscale=[
        [0, '#667eea'],
        [0.25, '#764ba2'],
        [0.5, '#f093fb'],
        [0.75, '#f5576c'],
        [1, '#ffa726']
    ],
    contours=dict(
        z=dict(show=True, usecolormap=True, highlightcolor="limegreen", project_z=True)
    )
)])

fig.update_layout(
    title=dict(
        text='LME銅先物カーブの時系列変化（3D）',
        font=dict(size=24, color=COLORS['dark'])
    ),
    scene=dict(
        xaxis_title='限月（月数）',
        yaxis_title='取引日',
        zaxis_title='価格 (USD/t)',
        camera=dict(eye=dict(x=1.5, y=-1.5, z=1.2))
    ),
    width=1000,
    height=700,
    template=plotly_template
)

fig.show()

## 2. 取引所間の価格比較（アニメーション付き）

In [ ]:
# 1番限のデータのみ抽出
first_month = futures_df[futures_df['TenorNumber'] == 1].copy()

# 日付でソート
first_month = first_month.sort_values('TradeDate')

# アニメーション付き折れ線グラフ
fig = px.line(
    first_month,
    x='TradeDate',
    y='SettlementPrice',
    color='ExchangeCode',
    title='取引所別 銅先物価格推移（1番限）',
    labels={
        'TradeDate': '取引日',
        'SettlementPrice': '価格',
        'ExchangeCode': '取引所'
    },
    color_discrete_map={
        'LME': COLORS['primary'],
        'SHFE': COLORS['secondary'],
        'CMX': COLORS['accent']
    },
    line_shape='spline'
)

fig.update_traces(mode='lines+markers', line_width=3, marker_size=4)

fig.update_layout(
    width=1000,
    height=500,
    hovermode='x unified',
    template=plotly_template,
    title_font_size=20,
    xaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgba(128,128,128,0.1)'),
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgba(128,128,128,0.1)')
)

# 注釈を追加
latest_prices = first_month.groupby('ExchangeCode').last()
for exchange, row in latest_prices.iterrows():
    fig.add_annotation(
        x=row['TradeDate'],
        y=row['SettlementPrice'],
        text=f"{exchange}: ${row['SettlementPrice']:,.0f}",
        showarrow=True,
        arrowhead=2,
        bgcolor="white",
        bordercolor=COLORS['dark'],
        borderwidth=1
    )

fig.show()

## 3. 限月間スプレッドのヒートマップ

In [ ]:
# 最新取引日のLMEデータ
latest_date = lme_data['TradeDate'].max()
latest_lme = lme_data[lme_data['TradeDate'] == latest_date].copy()

# スプレッドマトリックスの作成
tenors = sorted(latest_lme['TenorNumber'].unique())
spread_matrix = pd.DataFrame(index=tenors, columns=tenors)

price_dict = dict(zip(latest_lme['TenorNumber'], latest_lme['SettlementPrice']))

for i in tenors:
    for j in tenors:
        if i in price_dict and j in price_dict:
            spread_matrix.loc[i, j] = price_dict[j] - price_dict[i]

spread_matrix = spread_matrix.astype(float)

# カスタムカラーマップ
fig = go.Figure(data=go.Heatmap(
    z=spread_matrix.values,
    x=[f'M{i}' for i in spread_matrix.columns],
    y=[f'M{i}' for i in spread_matrix.index],
    colorscale=[
        [0, '#3498db'],
        [0.25, '#9b59b6'],
        [0.5, '#ecf0f1'],
        [0.75, '#e74c3c'],
        [1, '#c0392b']
    ],
    text=spread_matrix.values,
    texttemplate='%{text:.0f}',
    textfont={"size": 10},
    hoverongaps=False
))

fig.update_layout(
    title=dict(
        text=f'LME銅限月間スプレッド（{latest_date.strftime("%Y-%m-%d")}）',
        font=dict(size=20)
    ),
    xaxis_title='買い限月',
    yaxis_title='売り限月',
    width=800,
    height=700,
    template=plotly_template
)

fig.show()

# スプレッド統計
print("\n📊 スプレッド統計:")
print(f"最大バックワード: ${spread_matrix.min().min():,.0f}")
print(f"最大コンタンゴ: ${spread_matrix.max().max():,.0f}")

## 4. 出来高と建玉の分析

In [ ]:
# LMEの限月別出来高・建玉
lme_volume_oi = lme_data.groupby('TenorNumber').agg({
    'Volume': 'mean',
    'OpenInterest': 'mean'
}).reset_index()

# サブプロット
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('平均出来高', '平均建玉'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

# 出来高バー
fig.add_trace(
    go.Bar(
        x=[f'M{i}' for i in lme_volume_oi['TenorNumber']],
        y=lme_volume_oi['Volume'],
        marker=dict(
            color=lme_volume_oi['Volume'],
            colorscale='Viridis',
            line=dict(color='rgb(8,48,107)', width=1.5)
        ),
        text=lme_volume_oi['Volume'].round(0),
        textposition='outside'
    ),
    row=1, col=1
)

# 建玉バー
fig.add_trace(
    go.Bar(
        x=[f'M{i}' for i in lme_volume_oi['TenorNumber']],
        y=lme_volume_oi['OpenInterest'],
        marker=dict(
            color=lme_volume_oi['OpenInterest'],
            colorscale='Plasma',
            line=dict(color='rgb(8,48,107)', width=1.5)
        ),
        text=lme_volume_oi['OpenInterest'].round(0),
        textposition='outside'
    ),
    row=1, col=2
)

fig.update_layout(
    title=dict(
        text='LME銅先物 限月別流動性分析',
        font=dict(size=22)
    ),
    showlegend=False,
    width=1000,
    height=500,
    template=plotly_template
)

fig.update_xaxes(title_text="限月", row=1, col=1)
fig.update_xaxes(title_text="限月", row=1, col=2)
fig.update_yaxes(title_text="契約数", row=1, col=1)
fig.update_yaxes(title_text="契約数", row=1, col=2)

fig.show()

## 5. 期間構造の変化（レーダーチャート）

In [ ]:
# 複数時点の期間構造を比較
dates_to_compare = pd.date_range(end=latest_date, periods=4, freq='W')
colors_radar = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

fig = go.Figure()

for idx, date in enumerate(dates_to_compare):
    # 最も近い取引日を検索
    closest_date = lme_data.loc[(lme_data['TradeDate'] - date).abs().idxmin()]['TradeDate']
    
    date_data = lme_data[lme_data['TradeDate'] == closest_date]
    if len(date_data) > 0:
        # 価格を正規化（最初の限月を100として）
        base_price = date_data[date_data['TenorNumber'] == 1]['SettlementPrice'].values[0]
        date_data['NormalizedPrice'] = (date_data['SettlementPrice'] / base_price) * 100
        
        sorted_data = date_data.sort_values('TenorNumber')
        
        fig.add_trace(go.Scatterpolar(
            r=sorted_data['NormalizedPrice'],
            theta=[f'M{i}' for i in sorted_data['TenorNumber']],
            fill='toself',
            name=closest_date.strftime('%Y-%m-%d'),
            line=dict(color=colors_radar[idx], width=2),
            fillcolor=colors_radar[idx],
            opacity=0.4
        ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[95, 105]
        )
    ),
    showlegend=True,
    title=dict(
        text='期間構造の変化（正規化価格）',
        font=dict(size=20)
    ),
    width=800,
    height=600,
    template=plotly_template
)

fig.show()

## 6. ボラティリティ分析

In [ ]:
# 日次リターンの計算
lme_returns = lme_data.copy()
lme_returns = lme_returns.sort_values(['TenorNumber', 'TradeDate'])
lme_returns['DailyReturn'] = lme_returns.groupby('TenorNumber')['SettlementPrice'].pct_change() * 100

# 20日移動ボラティリティ
lme_returns['Volatility'] = lme_returns.groupby('TenorNumber')['DailyReturn'].transform(
    lambda x: x.rolling(window=20, min_periods=10).std() * np.sqrt(252)
)

# 最新のボラティリティ
latest_vol = lme_returns[lme_returns['TradeDate'] == latest_date]

# ボラティリティカーブ
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[f'M{i}' for i in sorted(latest_vol['TenorNumber'])],
    y=latest_vol.sort_values('TenorNumber')['Volatility'],
    mode='lines+markers',
    line=dict(color=COLORS['primary'], width=3),
    marker=dict(size=10, symbol='diamond'),
    fill='tonexty',
    fillcolor='rgba(46, 134, 171, 0.2)'
))

fig.update_layout(
    title=dict(
        text=f'限月別ボラティリティ（年率換算）- {latest_date.strftime("%Y-%m-%d")}',
        font=dict(size=20)
    ),
    xaxis_title='限月',
    yaxis_title='ボラティリティ (%)',
    width=900,
    height=500,
    template=plotly_template,
    yaxis=dict(tickformat='.1f')
)

fig.show()

print("\n📈 ボラティリティ統計:")
vol_stats = latest_vol.groupby('TenorNumber')['Volatility'].first()
print(f"最低ボラティリティ: M{vol_stats.idxmin()} ({vol_stats.min():.1f}%)")
print(f"最高ボラティリティ: M{vol_stats.idxmax()} ({vol_stats.max():.1f}%)")

## データベース接続のクローズ

In [ ]:
conn.close()
print("✅ データベース接続をクローズしました")